<a href="https://colab.research.google.com/github/luciaPi/Neural-networks-course/blob/main/3Trening_neuronova_siet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tréning neurónovej siete**

In [ ]:
import torch
from torch import nn
import pandas as pd
import torch.utils.data as data_utils
import torch.optim as optim

### **Dataset Študenti**

https://www.kaggle.com/datasets/rkiattisak/student-performance-in-mathematics

#### **1. Príprava dát**

In [ ]:
#!ls /root/.cache/kagglehub/datasets/rkiattisak/student-performance-in-mathematics/versions/3

import kagglehub

# Download latest version
path = kagglehub.dataset_download("rkiattisak/student-performance-in-mathematics")

print("Path to dataset files:", path)
data = pd.read_csv(path+"/exams.csv")

# zmenime "lunch" na 0-1 hodnoty
data['lunch'] = data['lunch'].replace({'free/reduced': 1, 'standard': 0})
# upravime aj "gender" a "parental level of education"
data['gender'] = data['gender'].replace({'female': 1, 'male': 0})
data['parental level of education'] = data['parental level of education'].replace({'some high school': 0,'high school':0,'some college':1,"associate's degree":2,"bachelor's degree":3,"master's degree":4})
# zmenime na 0-1 hodnoty stlpec "test preparation course"
data['test preparation course'] = data['test preparation course'].replace({'completed': 1, 'none': 0})
# vymazeme stlpec "race/ethnicity"
data =data.drop(columns=['race/ethnicity'])

data

In [ ]:
# vizualizujeme data
from matplotlib import pyplot as plt
import seaborn as sns

# Set up a 2x4 grid of subplots
fig, axes = plt.subplots(2, 3, figsize=(20, 10), constrained_layout=True)

# Plot 1: Grouped by 'parental_level_of_education'
data.groupby('parental level of education').size().plot(kind='barh', ax=axes[0, 0], color=sns.palettes.mpl_palette('Dark2'), title='parental level of education')
axes[0, 0].spines[['top', 'right']].set_visible(False)

# Plot 2: 'lunch'
data['lunch'].plot(kind='hist', bins=20, ax=axes[0, 1], title='lunch')
axes[0, 1].spines[['top', 'right']].set_visible(False)

# Plot 3: 'gender'
data['gender'].plot(kind='hist', bins=20, ax=axes[0, 2], title='gender')
axes[0, 2].spines[['top', 'right']].set_visible(False)

# Plot 4: 'test_preparation_course'
data['test preparation course'].plot(kind='hist', bins=20, ax=axes[1, 0], title='test preparation course')
axes[1, 0].spines[['top', 'right']].set_visible(False)

# Plot 5: 'math_score'
data['math score'].plot(kind='hist', bins=20, ax=axes[1, 1], title='math score')
axes[1, 1].spines[['top', 'right']].set_visible(False)

# Plot 6: Scatter plot of 'math_score' vs 'reading_score'
data.plot(kind='scatter', x='math score', y='reading score', s=32, alpha=0.8, ax=axes[1, 2], title='math score vs reading score')
axes[1, 2].spines[['top', 'right']].set_visible(False)

In [ ]:
# zobrazime data
data

In [ ]:
# vstupy
input_name1 = 'test preparation course'
input_name2 = 'reading score'
input_name3 = 'writing score'
inputs = data[[input_name1,input_name2,input_name3]].copy()
inputs

In [ ]:
# vystup
output_name = 'math score'
labels = data[output_name].copy()
labels

In [ ]:
# upravime data na tenzory
x_data = torch.tensor(inputs.values, dtype=torch.float32)
y_data = torch.tensor(labels.values, dtype=torch.float32)

print(x_data.shape, y_data.shape)
x_data[:5], y_data[:5]

**DataLoader** - načítava dáta z datasetu pre neurónovú sieť

**Batch** je dávka dát. Dáta modelu ukazujeme v **batchoch** (dávkach).

**batch size** = počet dát v batchi

In [ ]:
batch_size = 8
train_tensor = data_utils.TensorDataset(x_data, y_data)
train_dataloader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

In [ ]:
# prvy batch dat
next(iter(train_dataloader))

#### **2. Príprava neurónovej siete**

In [ ]:
# prioritne budeme trenovat na GPU karte
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
# postavime neuronovu siet

class StudentiModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(3, 3)  # torch.nn.Linear(in_features, out_features)
        self.linear2 = nn.Linear(3, 2)
        self.linear3 = nn.Linear(2, 1)

    def forward(self, x):  # x = vstup
        y123 = self.linear1(x)
        y45 = self.linear2(y123)
        y6 = self.linear3(y45)
        return y6

# vyrvorime neuronovu siet
model = StudentiModel().to(device).eval()
model

**Úlohy**
1. Koľko vrstiev má neurónová sieť?
2. Koľko neurónov je v jednotlivých vrstvách?
3. Nakreslite obrázok siete.

**Váhy siete**

In [ ]:
# vahy su inicializovane nahodne
for name, param in model.named_parameters():
  print(name, "-", param.detach().numpy())

#### **3. Ohodnotenie siete pred tréningom**

In [ ]:
# vyberte studenta
index_vstupu = ???
print("Vstup =",x_data[index_vstupu])
y = model(x_data[index_vstupu]).item()
print("Vystup =",y)
print("Pozadovany vystup =",labels[index_vstupu])

**Úloha**

 Vypočítajte chybu siete: \\
a) absolútnu chybu \\
b) štvorcovú chybu

In [ ]:
chyba1 = ???
print("Absolutna chyba =", chyba1)

In [ ]:
chyba2 = ???
print("Stvorcova chyba =", chyba2)

**Chyba celého datasetu**

In [ ]:
# vystup celeho datasetu
y = model(x_data).detach().numpy()[:,0]

# priemerna absolutna chyba (MAE=mean absolute error)
MAE = abs(y_data-y).mean().item()

# priemerna stvorcova chyba (MSE=mean squarred error)
MSE = ((y_data-y)**2).mean().item()

print("MAE:",MAE)
print("MSE:",MSE)

#### **4. Tréning neurónovej siete**

In [ ]:
model.train() # train() - nastavime model do treningoveho modu (opakom je eval() - evaluacny/netreningovy mod)

In [ ]:
# ukazkove vstupy a vystupy
inputs, labels = next(iter(train_dataloader))
inputs, labels

In [ ]:
# optimizer - optimalizuje parametre (vahy, bias) siete
optimizer = optim.SGD(model.parameters(), lr=0.00001, momentum=0.9)

In [ ]:
# Loss funkcia = chyba siete - trening sa ju snazi zlepsit
criterion = nn.MSELoss() # MSE=mean squarred error (priemerna stvorcova chyba)

**Epocha** – modelu ukážeme jedenkrát všetky trénovacie dáta.

**Loss funkcia** hodnotí kvalitu aktuálneho modelu (s aktuálnymi váhami). **Minimalizujeme** ju = chceme, aby mala čo najmenšiu hodnotu (aby bola **chyba** modelu čo najmenšia).

<u>**Tréning modelu (učenie)**</u>

1. Tréning prebieha v niekoľkých **epochách**.
2. Dáta modelu ukazujeme v **batchoch** (dávkach).
3. Po každom **batchi** sa upravia **váhy** modelu tak, aby bola jeho **chyba** čo najmenšia (= minimalizujeme loss funkciu).

<u>**Jeden krok tréningu (pre 1 batch)**</u>
1. Reštartujeme optimizer
*optimizer.zero_grad()*
2. Vypočítame výstup siete pre aktuálny batch dát
*y = model(inputs)*
3. Vypočítame chybu siete pre aktuálny batch dát
*loss = criterion(y, labels)*
4. Vypočítame, ako by sa mali upraviť váhy, aby sa chyba zmenšila
*loss.backward()*
5. Úprava váh siete
*optimizer.step()*

Hľadanie najlepšieho modelu (váh) s čo najmenšou chybou (hodnotou Loss funkcie).

<img src="https://media.licdn.com/dms/image/v2/D4D12AQEwBsLCCT39Ug/article-cover_image-shrink_600_2000/article-cover_image-shrink_600_2000/0/1673634305799?e=2147483647&v=beta&t=3ikrAgs8p0EGgX0Motjp9rJr9CSBZjXLCi6WoBwqKoI" width=45% align="left">
<img src="https://cs.hse.ru/data/2016/08/26/1121363361/moml.jpg" width=45% align="right">

In [ ]:
# pocet epoch
epochs = ???

In [ ]:
print("Start training")

absolute_errors = []
losses_squarred_errors = []

for epoch in range(epochs): # pocas jednej epochy prejdeme cely dataset

    # inicializujeme pre treningove statistiky
    running_loss_mse = 0.0
    running_mae = 0.0
    batches = 0

    # v cykle prejdeme vsetky data
    for i, data in enumerate(train_dataloader):
        inputs, labels = data
        batches+=1

        # reshape data
        inputs = inputs.view(-1,3)
        labels = labels.view(-1,1)

        # restartujeme optimizer
        optimizer.zero_grad()

        # vypocitame vystup siete pre aktualny batch dat
        y = model(inputs)

        # priemerna absolutna chyba (MAE=mean absolute error)
        MAE = abs(labels-y).mean()
        running_mae += MAE.item()
        # chyba siete - trening sa ju snazi zlepsit (MSE=mean squarred error)
        loss = criterion(y, labels)

        # uprava parametrov siete podla aktualnej chyby (loss)
        loss.backward()
        optimizer.step()

        running_loss_mse += loss.item()

    # priemerna chyba za celu epochu
    MSE = running_loss_mse/batches
    losses_squarred_errors.append(MSE)
    MAE = running_mae/batches
    absolute_errors.append(MAE)

    print('epoch {0}: MSE={1} MAE={2}'.format(epoch+1,MSE,MAE))

print("Finished training")

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(10,4),constrained_layout=True)
axes[0].plot(losses_squarred_errors, label='Training Loss MSE')
axes[1].plot(absolute_errors, label='MAE')
axes[0].set_xlabel('Epochs')
axes[1].set_xlabel('Epochs')
axes[0].set_title('Training Loss MSE')
axes[1].set_title('MAE')
plt.show()

#### **5. Ohodnotenie siete po tréningu**

In [ ]:
# nastavime model do evaluacneho (netreningoveho modu)
model.eval()

In [ ]:
# vyberte studenta
index_vstupu = ???
print("Vstup =",x_data[index_vstupu])
y = model(x_data[index_vstupu]).item()
print("Vystup =",y)
print("Pozadovany vystup =",labels[index_vstupu])
chyba1 = abs(y-labels[index_vstupu])
print("Absolutna chyba =", chyba1)
chyba2 = (y-labels[index_vstupu])**2
print("Stvorcova chyba =", chyba2)

**Chyba celého datasetu**

In [ ]:
# vystup celeho datasetu
y = model(x_data).detach().numpy()[:,0]

# priemerna absolutna chyba (MAE=mean absolute error)
MAE = abs(y_data-y).mean().item()

# priemerna stvorcova chyba (MSE=mean squarred error)
MSE = ((y_data-y)**2).mean().item()

print("MAE:",MAE)
print("MSE:",MSE)

Neural Network Playground

https://playground.tensorflow.org/#activation=linear&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=10&networkShape=4,2&seed=0.92143&showTestData=false&discretize=false&percTrainData=70&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=regression&initZero=false&hideText=false